In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Point, LineString, MultiPoint
import sqlite3

In [2]:
filename = 'inst_geopackage.gpkg'

### Create a GeoPackage file with multiple layers

In [ ]:
help(MultiPoint)

In [ ]:
help(LineString)

In [ ]:
np.random.uniform(-100, 100, (num_points, 2))

In [92]:
filename = "testing_geom.gpkg"

In [93]:
# Create one dataframe with points
for ii in range(3):
    num_points = 10
    coords = np.random.uniform(-100, 100, (num_points, 2))

    points = [Point(xx, yy) for xx, yy in coords]
    gdf = gpd.GeoDataFrame(points, columns=['geometry'])

    # Add fields to the GeoDataFrame
    # QUESTION: Are these required? Will it auto-generate unique IDs if I leave that out?
    #gdf['id'] = [1, 2]
    gdf['name'] = ['Point {}'.format(ii) for ii in np.arange(num_points)]
    gdf['metadata'] = ['Testing metadata for point {}'.format(ii) for ii in np.arange(num_points)]

    gdf.crs = 'EPSG:3031' 
    gdf.to_file(filename, driver='GPKG', layer='points{:02d}'.format(ii))


In [94]:
gdf.values[gdf.index[3]]

array([<POINT (72.597 94.213)>, 'Point 3', 'Testing metadata for point 3'],
      dtype=object)

In [5]:
institution_frame = pd.DataFrame({'Institution': ['AWI', 'BAS', 'CRESIS', 'KOPRI', 'LDEO', 'STANFORD', 'UTIG']})

In [6]:
import sqlalchemy
with sqlite3.connect(filename) as conn:
    institution_frame.to_sql("institutions", conn, if_exists="replace")

In [7]:
gdf.index

RangeIndex(start=0, stop=10, step=1)

In [8]:
# Try to create another with line strings
linestrings = []
for ii in range(3):
    num_lines = 3
    lines = [LineString(np.random.uniform(-100, 100, (10, 2))) for _ in np.arange(num_lines)]
    gdf = gpd.GeoDataFrame(lines, columns=['geometry'])

    # Add fields to the GeoDataFrame
    #gdf['id'] = [1, 2]
    gdf['name'] = ['Line {}'.format(ii) for ii in np.arange(num_lines)]
    gdf.crs = 'EPSG:3031'  
    gdf.to_file(filename, driver='GPKG', layer='lines{:02d}'.format(ii))


In [9]:
# And yet another with MultiPoints.
for ii in range(3):
    num_points = 1
    rand_pts = np.random.uniform(-100, 100, (10, 2))
    print(rand_pts.shape)
    points = [MultiPoint(rand_pts) for _ in np.arange(num_points)]
    print(ii, points)
    gdf = gpd.GeoDataFrame(points, columns=['geometry'])

    # Add fields to the GeoDataFrame
    #gdf['id'] = [1, 2]
    gdf['name'] = ['MP {}'.format(ii) for ii in np.arange(num_points)]
    gdf.crs = 'EPSG:3031'  
    gdf.to_file(filename, driver='GPKG', layer='mp{:02d}'.format(ii))


(10, 2)
0 [<MULTIPOINT (-72.925 68.457, -75.172 14.543, -92.553 30.868, 19.21 71.311, 6...>]
(10, 2)
1 [<MULTIPOINT (45.902 99.333, 70.914 -28.628, 46.591 81.244, 70.957 62.563, -6...>]
(10, 2)
2 [<MULTIPOINT (-26.268 50.301, 69.452 -55.123, -90.608 77.305, -31.47 40.084, ...>]


In [101]:
help(gdf.to_file)

Help on method to_file in module geopandas.geodataframe:

to_file(filename, driver=None, schema=None, index=None, **kwargs) method of geopandas.geodataframe.GeoDataFrame instance
    Write the ``GeoDataFrame`` to a file.
    
    By default, an ESRI shapefile is written, but any OGR data source
    supported by Fiona can be written. A dictionary of supported OGR
    providers is available via:
    
    >>> import fiona
    >>> fiona.supported_drivers  # doctest: +SKIP
    
    Parameters
    ----------
    filename : string
        File path or file handle to write to. The path may specify a
        GDAL VSI scheme.
    driver : string, default None
        The OGR format driver used to write the vector file.
        If not specified, it attempts to infer it from the file extension.
        If no extension is specified, it saves ESRI Shapefile to a folder.
    schema : dict, default None
        If specified, the schema dictionary is passed to Fiona to
        better control how the fi

### Trying to create one with the bedmap data

In [10]:
import pandas as pd
def load_xy(filepath):
    '''Open file in the BEDMAP CVS format and extract just lat long from it'''
    skip_lines = count_skip_lines(filepath)
    data = pd.read_csv(filepath, skiprows=skip_lines)

    x_index = [col for col in data.columns if 'easting' in col][0]
    y_index = [col for col in data.columns if 'northing' in col][0]
    xx = data[x_index]
    yy = data[y_index]
    return np.array(xx), np.array(yy)

In [11]:
def count_skip_lines(filepath):
    """
    Count how many comment lines are at the start of a CSV file.

    QGS's vector layer API doesn't appear to have a setting that says
    "ignore all lines starting with a '#'", even though the GUI seems
    to auto-detect that and fill in the number.
    """
    skip_lines = 0
    for line in open(filepath, 'r'):
        if line.startswith("#"):
            skip_lines += 1
        else:
            break
    return skip_lines

In [12]:
def create_gpkg_layer(layer_name, gpkg_filepath, csv_filepaths, geometry_names, color):
    """"
    Add data in the CSV to the Geopackage, and create a QGIS layer with
    that as the data backend.

    We can't just directly drag the Geopackage into QGIS because:
    1) I don't see how to group layers within the GeoPackage file
    2) Programmatically styling the geopackage is awkward. I've only
      figured out how to do it when running in the QGIS console.
    """
    print("Called create_gpkg_layer!")
    print("layer_name = {}".format(layer_name))
    print("gpkg_filepath = {}".format(gpkg_filepath))
    print("csv_filepaths = {}".format(csv_filepaths))
    print("geometry_names = {}".format(geometry_names))
    print("color = {}".format(color))
    # Add a layer with these features to the GeoPackage
    geometries = []
    for filepath in csv_filepaths:
        xx, yy = load_xy(filepath)
        coords = np.array([[x1, y1] for x1, y1 in zip(xx, yy)])

        print(coords.shape)
        points = MultiPoint(coords)
        #print(points)
        geometries.append(points)
        print("ln 266")
    print(geometries)
    gdf = gpd.GeoDataFrame(geometries, columns=['geometry'])
    print("ln 267")
    gdf['name'] = geometry_names
    print("ln 269")
    gdf.crs = 'EPSG:3031'
    
    gdf.to_file(gpkg_filepath, driver='GPKG', layer=layer_name)
    

In [13]:
csv_filepaths = ["/Users/lindzey/RadarData/targ/ANTARCTIC/BEDMAP/AWI/AWI_1994_DML1_AIR_BM2.csv"]
gpkg_filepath = "test.gpkg"
geometry_names = ["AWI_1994_DML1"]
layer_name = "AWI_1994_DML1"
salmon = "251,154,153,255"

layer_name = "1994_DML1"
gpkg_filepath = "/Users/lindzey/RadarData/targ/qiceradar_index.gpkg"
csv_filepaths = ['/Users/lindzey/RadarData/targ/ANTARCTIC/BEDMAP/AWI/AWI_1994_DML1_AIR_BM2.csv']
geometry_names = ['1994_DML1']
color = "251,154,153,255"


create_gpkg_layer(layer_name, gpkg_filepath, csv_filepaths, geometry_names, color)
# WTF. I copy-and-pasted failing code, and it works....?

Called create_gpkg_layer!
layer_name = 1994_DML1
gpkg_filepath = /Users/lindzey/RadarData/targ/qiceradar_index.gpkg
csv_filepaths = ['/Users/lindzey/RadarData/targ/ANTARCTIC/BEDMAP/AWI/AWI_1994_DML1_AIR_BM2.csv']
geometry_names = ['1994_DML1']
color = 251,154,153,255
(22578, 2)
ln 266
[<MULTIPOINT (-285560.755 1557735.604, -285182.186 1557356.256, -284979.372 1...>]
ln 267
ln 269


In [ ]:
help(gdf.to_file)

### Can I get multiple groups loaded from a geopackage file? 

I'd like to be able to group layers by institution ...

Otherwise, I guess I could just publish one gpkg for each provider. 

OR, manually create the qlr file using the gpkg as the backend? If I look at a single layer, I see:
```
layer.source()

'/Users/lindzey/Documents/QIceRadar/radar_wrangler/code/data_exploration/my_geopackage.gpkg|layername=lines00'
```
Suggesting that I could load it like I've been doing, using that as the URI.
Then I'd have to figure out symbology, yet again ... 


### What about styling?

ChatGPT really struggled here, repeatedly telling me to access nonexistant methods.

I got partway there in python, but then segfaulted: 
(not compatible with jupyter-lab, since my qgis install uses a different python)
```
from qgis.core import QgsApplication, QgsVectorLayer
from PyQt5.QtGui import QColor

qgs = QgsApplication([],False)

qgs.initQgis()
layer = QgsVectorLayer("my_geopackage.gpkg", "lines02", "ogr")            
layer.renderer().symbol().setColor(QColor("red"))

#layer.renderer().setSymbol(symbol)
#layer.saveDefaultStyle()
```

It might be useful to look at {save,load}NamedStyle et al.
saveStyleToDatabase seems likely to be the call that the GUI uses, given that the fields match exactly. 
```
saveStyleToDatabase(...) method of qgis._core.QgsVectorLayer instance
    saveStyleToDatabase(self, name: str, description: str, useAsDefault: bool, uiFileContent: str) -> str
    Saves named and sld style of the layer to the style table in the db.
    
    :param name: Style name
    :param description: A description of the style
    :param useAsDefault: Set to ``True`` if style should be used as the default style for the layer
    :param uiFileContent:
(END)
```

Unfortunately, that dies with: 
```
>>> layer.saveStyleToDatabase('test_red', 'testing, testing', True, '')
libc++abi: Pure virtual function called!
zsh: abort      python3
```

I tried using the SQLite DB Browser to create a new row in the layer_styles table, but that didn't work to set style for that layer, even though I copied it directly. 

I tried loading it via the dialog (rather than implicitly by drag-and-drop), and actually got a more informative error:

`The retrieved style is not a valid named style. Error message: Root <qgis> element could not be found`

Maybe I actually needed the 10k charactter styleQML field too, even though earlier experiments suggested styleSLD was enough?

There's also `{save,load}SldStyle`...

If I do it from *within* the Python Console in QGIS, this works:
```
layer = iface.activeLayer()
layer.renderer().symbol().setColor(QColor("cyan"))
layer.triggerRepaint()
layer.saveStyleToDatabase('test_cyan', '', True, '')
```

OK, so next attempt will be a script that goes through and sets the style for each layer. 

* Q1: Can I have a table of styles, and only specify style_name for a layer? Or do I have to have a separate row in the style_table for each layer?
* Q2: Assuming that I have to assign styles in a separate step from the layer creation, how can I programmatically access metadata indicating what color a layer whould be?
```
ss = layer.source().split('|')[0]

ss = QgsLineSymbol()
ss.setColor(QColor("red"))
rr = QgsSingleSymbolRenderer(ss)
layer.setRenderer(rr)

However, neither of the "save" commands I can find seem to work!
layer.saveNamedStyle("my_geopackage.gpkg")
layer.saveStyleToDatabase('test_red', '', True, '')
```

TO iterate through the layers of the file one I've dragged the GeoPackage into QGIS:
```
layer_tree_view = iface.layerTreeView()
nodes = layer_tree_view.selectedNodes()
node = nodes[0]
for child in nodes.children():
    ss = child.layer().renderer().symbol()
    ss.setColor(QColor("cyan"))
    child.layer().triggerRepaint()
```
This gets them all cyan, though the key in the ToC doesn't update. Close the layer and QGIS will write changes to teh gpkg file. Re-drag them in, and everything is cyan.

### Using sqlite3 to look at underlying structure of database

In [14]:
gpt = "chatgpt_geopackage.gpkg"

In [25]:
index = "/Users/lindzey/RadarData/targ/qiceradar_index.gpkg"

In [15]:
foo = "my_geopackage2.gpkg-shm"

In [16]:
def list_tables(filename):
    with sqlite3.connect(filename) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT name FROM sqlite_master WHERE type="table"')
        tables = cursor.fetchall()
        for table in tables:
            tablename = table[0]
            print("{}:".format(tablename))
            cursor.execute('PRAGMA table_info({})'.format(tablename))
            #for row in cursor:
            #    print(row)
            fields = [row[1] for row in cursor]
            print(fields)


In [17]:
list_tables(foo)

In [ ]:
list_tables(filename)

In [ ]:
list_tables(index)

In [60]:
def read_table(filename, tablename, maxrows=None):
    with sqlite3.connect(filename) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM {}'.format(tablename))
        print("{}:".format(tablename))
        numrows = 0
        for row in cursor:
            #print(row)
            #output = [type(elem) for elem in row]
            output = [elem if type(elem) is not bytes else "blob"
                      for elem in row]
            print(output)
            numrows += 1
            if maxrows is not None and numrows >= maxrows:
                break

In [61]:
read_table(index, 'gpkg_contents', 3)

gpkg_contents:
['1994_DML1', 'features', '1994_DML1', '', '2023-01-03T20:46:15.958Z', -709554.6979392086, 978681.179640682, -198292.9658415384, 1714829.881174039, 3031]
['AWI_1994_DML1', 'features', 'AWI_1994_DML1', '', '2023-01-04T16:53:38.227Z', -709554.6979392086, 978681.179640682, -198292.9658415384, 1714829.881174039, 3031]
['AWI_1995_DML2', 'features', 'AWI_1995_DML2', '', '2023-01-04T16:53:38.691Z', -430662.1741941844, 1398761.277751747, 767915.2728331031, 2156149.372818692, 3031]


In [62]:
# I think these are the ones that I need
read_table(index, 'gpkg_geometry_columns', 5)

gpkg_geometry_columns:
['1994_DML1', 'geom', 'MULTIPOINT', 3031, 0, 0]
['AWI_1994_DML1', 'geom', 'MULTIPOINT', 3031, 0, 0]
['AWI_1995_DML2', 'geom', 'MULTIPOINT', 3031, 0, 0]
['AWI_1996_DML3', 'geom', 'MULTIPOINT', 3031, 0, 0]
['AWI_1997_DML4', 'geom', 'MULTIPOINT', 3031, 0, 0]


In [63]:
read_table(index, 'AWI_1994_DML1', 1)

AWI_1994_DML1:
[1, 'blob', 'AWI_1994_DML1', None, 'AWI', None, None, '1994_DML1', 'u']


In [64]:
read_table(filename, 'gpkg_geometry_columns')

gpkg_geometry_columns:
['points00', 'geom', 'POINT', 3031, 0, 0]
['points01', 'geom', 'POINT', 3031, 0, 0]
['points02', 'geom', 'POINT', 3031, 0, 0]
['lines00', 'geom', 'LINESTRING', 3031, 0, 0]
['lines01', 'geom', 'LINESTRING', 3031, 0, 0]
['lines02', 'geom', 'LINESTRING', 3031, 0, 0]
['mp00', 'geom', 'MULTIPOINT', 3031, 0, 0]
['mp01', 'geom', 'MULTIPOINT', 3031, 0, 0]
['mp02', 'geom', 'MULTIPOINT', 3031, 0, 0]


In [65]:
read_table(filename, 'points00')

points00:
[1, 'blob', 'Point 0']
[2, 'blob', 'Point 1']
[3, 'blob', 'Point 2']
[4, 'blob', 'Point 3']
[5, 'blob', 'Point 4']
[6, 'blob', 'Point 5']
[7, 'blob', 'Point 6']
[8, 'blob', 'Point 7']
[9, 'blob', 'Point 8']
[10, 'blob', 'Point 9']


In [70]:
def list_columns(filename, tablename):
    with sqlite3.connect(filename) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM {}'.format(tablename))
        columns = [elem[0] for elem in cursor.description]
        print(columns)


In [71]:
list_columns(index, 'gpkg_geometry_columns')

['table_name', 'column_name', 'geometry_type_name', 'srs_id', 'z', 'm']


In [73]:
list_columns(index, 'AWI_1994_DML1')

['fid', 'geom', 'name', 'uri', 'institution', 'granule', 'segment', 'campaign', 'availability']


In [79]:
def list_granule_availability(filename, tablename):
    with sqlite3.connect(filename) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.execute('SELECT * FROM {}'.format(tablename))
        data = {row['name']: row['availability'] for row in cursor}
        print(data)

In [80]:
list_granule_availability(index, 'AWI_1994_DML1')

{'AWI_1994_DML1': 'u'}


In [82]:
foo = set([1,2,3])

AttributeError: 'set' object has no attribute 'size'

In [87]:


    institutions = set()
    granules = set()
    with sqlite3.connect(gpkg_filepath) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.execute('SELECT * FROM {}'.format('gpkg_geometry_columns'))
        for row in cursor:
            granules.add(row['table_name'])  # I think this is also the primary key

        for granule in granules:
            print("Processing {}".format(granule))
            cursor = conn.execute("SELECT * FROM '{}'".format(granule))
            # NOTE: With the current database design, these tables only have one row.
            for row in cursor:
                institutions.add(row['institution'])

    print("{} granules from {} institutions".format(len(granules), len(institutions)))

Processing NASA_2010_ICEBRIDGE
Processing AWI_2002_DML8
Processing RNRF_2009_RAEap5
Processing NIPR_2012_JARE54
Processing UTIG_1991_CASERTZ
Processing RNRF_2004_Mirny-Vostok
Processing AWI_2003_DML9
Processing INGV_1997_Talos-Dome
Processing RNRF_2003_AMSap5
Processing NIPR_2018_JARE60
Processing AWI_2018_DML-Coast
Processing RNRF_2014_RAE
Processing ULB_2012_ICECON
Processing UTIG_1999_SOAR-LVS-WLK
Processing AWI_2004_DML10
Processing UTIG_2008_ICECAP
Processing AWI_1998_DML5
Processing RNRF_2015_RAE
Processing CRESIS_2009_AntarcticaTO
Processing STOLAF_2001_ITASE-Ellsworth
Processing KOPRI_2017_KRT1
Processing RNRF_2006_Komsom-Vostok
Processing RNRF_2003_48RAEap5
Processing UWASHINGTON_2018_South-Pole-Lake
Processing PRIC_2018_CHA4
Processing AWI_2013_GEA-IV
Processing UTIG_1998_West-Marie-Byrd-Land
Processing BAS_2007_Rutford
Processing STOLAF_2002_ITASE-Byrd-South-Pole
Processing UTIG_2013_GIMBLE
Processing RNRF_2011_RAE
Processing AWI_1994_DML1
Processing BAS_2001_MAMOG
Processin

In [88]:
institutions

{'AWI',
 'BAS',
 'BEDMAP1',
 'BGR',
 'CECS',
 'CRESIS',
 'INGV',
 'KOPRI',
 'LDEO',
 'NASA',
 'NIPR',
 'NPI',
 'PRIC',
 'RNRF',
 'STANFORD',
 'STOLAF',
 'UCANTERBURY',
 'ULB',
 'UTIG',
 'UWASHINGTON'}

In [89]:
granules

{'AWI_1994_DML1',
 'AWI_1995_DML2',
 'AWI_1996_DML3',
 'AWI_1997_DML4',
 'AWI_1998_DML5',
 'AWI_2000_DML6',
 'AWI_2001_DML7',
 'AWI_2002_DML8',
 'AWI_2003_DML9',
 'AWI_2004_DML10',
 'AWI_2005_ANTSYSO',
 'AWI_2007_ANTR',
 'AWI_2013_GEA-IV',
 'AWI_2014_Recovery-Glacier',
 'AWI_2015_GEA-DML',
 'AWI_2016_OIR',
 'AWI_2018_ANIRES',
 'AWI_2018_DML-Coast',
 'AWI_2018_JURAS',
 'AWI_2019_JURAS',
 'BAS_1994_Evans',
 'BAS_1998_Dufek',
 'BAS_2001_Bailey-Slessor',
 'BAS_2001_MAMOG',
 'BAS_2002_TORUS',
 'BAS_2007_Lake-Ellsworth',
 'BAS_2007_Rutford',
 'BAS_2007_TIGRIS',
 'BAS_2008_Lake-Ellsworth',
 'BAS_2009_FERRIGNO',
 'BAS_2010_PIG',
 'BAS_2011_Adelaide',
 'BAS_2012_Castle',
 'BAS_2013_ISTAR',
 'BAS_2018_Thwaites',
 'BAS_2019_Thwaites',
 'BEDMAP1_BEDMAP1',
 'BGR_1999_GANOVEX-VIII-Matusevich',
 'BGR_1999_GANOVEX-VIII-Mertz',
 'BGR_2002_PCMEGA',
 'CECS_2006_Subglacial-Lake-CECs',
 'CRESIS_2009_AntarcticaTO',
 'CRESIS_2009_Thwaites',
 'CRESIS_2013_Siple-Coast',
 'INGV_1997_ITASE',
 'INGV_1997_Talos-Do

## Figure out geometry type

In [105]:
gpkg_filepath = "/Users/lindzey/RadarData/targ/qiceradar_index.gpkg"

In [108]:
campaign = "STANFORD_SPRI_NSF_TUD"
#campaign = "UTIG_2004_AGASEA"
with sqlite3.connect(gpkg_filepath) as conn:
    conn.row_factory = sqlite3.Row
    cursor = conn.execute("SELECT * FROM {}".format('gpkg_geometry_columns'))
    for row in cursor:
        if row['table_name'] == campaign:
            print("Campaign {} has geometry {}".format(campaign, row['geometry_type_name']))
            break

Campaign STANFORD_SPRI_NSF_TUD has geometry LINESTRING
